In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
import math
import re
import subprocess
from os import listdir
from os.path import isfile, join

In [34]:
import matplotlib as mpl 
mpl.use('Agg')
mpl.rcParams['pdf.fonttype'] = 42
import seaborn as sns
import matplotlib.pyplot as plt
plt.close("all")
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams["font.family"] = 'sans-serif'
mpl.rcParams['font.size'] = 8
sns.set(style="ticks")

In [2]:
cd /Users/davidy/jamboree20crispr/ontarget_analysis/220427_distance_optimization

/Users/davidy/jamboree20crispr/ontarget_analysis/220427_distance_optimization


In [10]:
### Format output from guidescan 
guidescan_df = pd.read_csv('dhs_h3k27ac_peak_guidescan_results.csv')
guidescan_df['guide_coordinates'] = guidescan_df['Coordinates'].str.split(':').str[-1]
guidescan_df['pam_coordinate'] = np.where(guidescan_df['Strand']=='+',
                                          guidescan_df['guide_coordinates'].str.split('-').str[1].astype(int)-4,
                                         guidescan_df['guide_coordinates'].str.split('-').str[0].astype(int)+3)


,Region-name,gRNA-ID,gRNA-Seq,Target-Seq,PAM,Number of off-targets,Off-target summary,Cutting efficiency,Specificity,Rank,Coordinates,Strand,Annotations,guide_coordinates,pam_coordinate
0,chr8:128076892-128077425,chr8:128076892-128077425.0,CAGGGGAATACCCTTTGCCG,CGGCAAAGGGTATTCCCCTG,NGG,101,2:0 | 3:2,0.622122,0.739710,1,NC_000008.11:128076974-128076996,-,NaN,128076974-128076996,128076977
1,chr8:128076892-128077425,chr8:128076892-128077425.1,TTGTGGTGGATCCACGGCAA,TTGTGGTGGATCCACGGCAA,NGG,103,2:0 | 3:4,0.539133,0.836607,2,NC_000008.11:128076963-128076985,+,NaN,128076963-128076985,128076981
2,chr8:128076892-128077425,chr8:128076892-128077425.2,CACTTAACTAAGGGGTGTTC,GAACACCCCTTAGTTAAGTG,NGG,103,2:0 | 3:4,0.338728,0.801579,3,NC_000008.11:128077271-128077293,-,NaN,128077271-128077293,128077274
3,chr8:128076892-128077425,chr8:128076892-128077425.3,TTTCCGCTGCTGAGCTCGCT,TTTCCGCTGCTGAGCTCGCT,NGG,104,2:0 | 3:5,0.363218,0.785925,4,NC_000008.11:128077342-128077364,+,NaN,128077342-128077364,128077360
4,chr8:128076892-128077425,chr8:128076892-128077425.4,TTTTCCGCTGCTGAGCTCGC,TTTTCCGCTGCTGAGCTCGC,NGG,106,2:0 | 3:7,0.338864,0.370861,5,NC_000008.11:128077341-128077363,+,NaN,128077341-128077363,128077359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,chr5:88836308-88836965,chr5:88836308-88836965.65,ATGAAATGAAAAACTGGAGT,ACTCCAGTTTTTCATTTCAT,NGG,220,2:4 | 3:117,0.559068,0.058196,66,NC_000005.10:88836446-88836468,-,NaN,88836446-88836468,88836449
3043,chr5:88836308-88836965,chr5:88836308-88836965.66,AAGGTGATGAAATGAAAAAC,GTTTTTCATTTCATCACCTT,NGG,227,2:7 | 3:121,0.364517,0.076850,67,NC_000005.10:88836452-88836474,-,NaN,88836452-88836474,88836455
3044,chr5:88836308-88836965,chr5:88836308-88836965.67,TCTGTCCCAGGCTGTTTTCA,TCTGTCCCAGGCTGTTTTCA,NGG,234,2:3 | 3:132,0.403843,0.046708,68,NC_000005.10:88836706-88836728,+,NaN,88836706-88836728,88836724
3045,chr5:88836308-88836965,chr5:88836308-88836965.68,TGTCAAAGAGCTAGGAATAG,TGTCAAAGAGCTAGGAATAG,NGG,258,2:0 | 3:159,0.513124,0.057161,69,NC_000005.10:88836528-88836550,+,NaN,88836528-88836550,88836546


In [32]:
### Get summit information

# Get the summit positions via intersection
peak_file = 'dhs_h3k27ac_elements_dhs_peaks.bed'
fopen = open('dhs_peak_summits.bed', 'w')
subprocess.call(['bedtools','intersect','-a',peak_file,'-b','/Users/davidy/misc_resources/chip/results/macs2/dhs_k562_hg38_optimal_peak.narrowPeak.bed', '-wa','-wb'],stdout=fopen)
fopen.close()

# Only pick the summit with the strongest epigenetic signal
peak_intersect_df = pd.read_csv('dhs_peak_summits.bed', sep = '\t', header=None, names = ['chr1','start1','end1','chr2','start2','end2','blank1','blank2','blank3','signal1','signal2','signal3','summit'], usecols=['chr2','start2','end2','signal1','summit'])
peak_intersect_df['dhs_peak'] = peak_intersect_df['chr2'].astype(str) + ':' + peak_intersect_df['start2'].astype(str) + '-' + peak_intersect_df['end2'].astype(str)
peak_intersect_df = peak_intersect_df.loc[peak_intersect_df.groupby('dhs_peak')['signal1'].idxmax()]
peak_intersect_df['summit_position'] = peak_intersect_df['start2']+peak_intersect_df['summit']
display(peak_intersect_df)

### Record peak:summit position dict
# Key = DHS peak in proper :- format, Value = Absolute summit position, based on strongest epigenetic signal
summit_position_dict = dict(zip(peak_intersect_df['dhs_peak'],peak_intersect_df['summit_position'])) 







,chr2,start2,end2,signal1,summit,dhs_peak,summit_position
26,chr11,33881952,33882395,4.48770,122,chr11:33881952-33882395,33882074
38,chr11,33936857,33937690,4.22598,576,chr11:33936857-33937690,33937433
31,chr11,33941604,33942092,3.64260,299,chr11:33941604-33942092,33941903
33,chr11,33942939,33943233,3.50261,178,chr11:33942939-33943233,33943117
6,chr11,33944500,33945221,5.30747,393,chr11:33944500-33945221,33944893
4,chr11,61834010,61835352,6.41215,812,chr11:61834010-61835352,61834822
13,chr11,61869482,61870486,4.92284,463,chr11:61869482-61870486,61869945
16,chr11,61870793,61871589,4.92096,541,chr11:61870793-61871589,61871334
74,chr5,88836308,88836965,10.20799,395,chr5:88836308-88836965,88836703
25,chr5,89055045,89055589,8.74388,184,chr5:89055045-89055589,89055229


In [33]:
### Format guidescan df to include distance from guide to its peaks summit
guidescan_df['distance_to_summit'] = guidescan_df['pam_coordinate'] - guidescan_df['Region-name'].map(summit_position_dict)
display(guidescan_df)



,Region-name,gRNA-ID,gRNA-Seq,Target-Seq,PAM,Number of off-targets,Off-target summary,Cutting efficiency,Specificity,Rank,Coordinates,Strand,Annotations,guide_coordinates,pam_coordinate,distance_to_summit
0,chr8:128076892-128077425,chr8:128076892-128077425.0,CAGGGGAATACCCTTTGCCG,CGGCAAAGGGTATTCCCCTG,NGG,101,2:0 | 3:2,0.622122,0.739710,1,NC_000008.11:128076974-128076996,-,NaN,128076974-128076996,128076977,-164
1,chr8:128076892-128077425,chr8:128076892-128077425.1,TTGTGGTGGATCCACGGCAA,TTGTGGTGGATCCACGGCAA,NGG,103,2:0 | 3:4,0.539133,0.836607,2,NC_000008.11:128076963-128076985,+,NaN,128076963-128076985,128076981,-160
2,chr8:128076892-128077425,chr8:128076892-128077425.2,CACTTAACTAAGGGGTGTTC,GAACACCCCTTAGTTAAGTG,NGG,103,2:0 | 3:4,0.338728,0.801579,3,NC_000008.11:128077271-128077293,-,NaN,128077271-128077293,128077274,133
3,chr8:128076892-128077425,chr8:128076892-128077425.3,TTTCCGCTGCTGAGCTCGCT,TTTCCGCTGCTGAGCTCGCT,NGG,104,2:0 | 3:5,0.363218,0.785925,4,NC_000008.11:128077342-128077364,+,NaN,128077342-128077364,128077360,219
4,chr8:128076892-128077425,chr8:128076892-128077425.4,TTTTCCGCTGCTGAGCTCGC,TTTTCCGCTGCTGAGCTCGC,NGG,106,2:0 | 3:7,0.338864,0.370861,5,NC_000008.11:128077341-128077363,+,NaN,128077341-128077363,128077359,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,chr5:88836308-88836965,chr5:88836308-88836965.65,ATGAAATGAAAAACTGGAGT,ACTCCAGTTTTTCATTTCAT,NGG,220,2:4 | 3:117,0.559068,0.058196,66,NC_000005.10:88836446-88836468,-,NaN,88836446-88836468,88836449,-254
3043,chr5:88836308-88836965,chr5:88836308-88836965.66,AAGGTGATGAAATGAAAAAC,GTTTTTCATTTCATCACCTT,NGG,227,2:7 | 3:121,0.364517,0.076850,67,NC_000005.10:88836452-88836474,-,NaN,88836452-88836474,88836455,-248
3044,chr5:88836308-88836965,chr5:88836308-88836965.67,TCTGTCCCAGGCTGTTTTCA,TCTGTCCCAGGCTGTTTTCA,NGG,234,2:3 | 3:132,0.403843,0.046708,68,NC_000005.10:88836706-88836728,+,NaN,88836706-88836728,88836724,21
3045,chr5:88836308-88836965,chr5:88836308-88836965.68,TGTCAAAGAGCTAGGAATAG,TGTCAAAGAGCTAGGAATAG,NGG,258,2:0 | 3:159,0.513124,0.057161,69,NC_000005.10:88836528-88836550,+,NaN,88836528-88836550,88836546,-157


In [54]:
### Plot specificity vs distance to summit

g1 = sns.jointplot(data=guidescan_df, x='distance_to_summit',y='Specificity',alpha=0.5, linewidth=0)
outplot = 'DHSPeak_gRNA_CFD_vs_SummitDistance.pdf'
g1.ax_marg_x.set_xlim(-750, 750)
g1.ax_marg_y.set_ylim(0, 1)
plt.savefig(outplot)
plt.close('all')



In [51]:
### Average CFD across bins 
# First create bin ranges, arbitrariy set to (-750,750), in steps of 50 bp
lod = []

for i in range(-750,750,50):
    # Get rows where distance to summit is within bin range
    distance_bin = np.mean([i,i+50]) # distance_bin is the middle of the 50 bp window
    bin_avg_specificty = np.mean(guidescan_df.loc[(guidescan_df['distance_to_summit']>i) & (guidescan_df['distance_to_summit']<i+50)]['Specificity'])
    adf = {'bin':distance_bin,
          'avg_cfd':bin_avg_specificty}
    lod.append(adf)

bin_avg_cfd_df = pd.DataFrame(lod)
display(bin_avg_cfd_df)

g2 = sns.jointplot(data=bin_avg_cfd_df, x='bin',y='avg_cfd',alpha=0.5)
outplot = 'DHSPeak_gRNA_CFD_vs_SummitDistance_50bpBinAvgCFD.pdf'
g2.ax_marg_x.set_xlim(-750, 750)
g2.ax_marg_y.set_ylim(0, 1)
plt.savefig(outplot)
plt.close('all')


# Note, this is just a horizontal line and doesnt support our point.

,bin,avg_cfd
0,-725.0,0.142204
1,-675.0,0.238327
2,-625.0,0.237558
3,-575.0,0.252458
4,-525.0,0.309235
5,-475.0,0.188257
6,-425.0,0.257678
7,-375.0,0.267339
8,-325.0,0.231860
9,-275.0,0.210398
